# Librerias

In [275]:
from text_detection import get_aws_analyze_document

In [276]:
import os
import numpy as np
import time
import pandas as pd
import datetime
import re
import boto3
import matplotlib.pyplot as plt
import cv2
import spacy
from unicodedata import normalize
import boto3

# Funciones

In [277]:
###Variables
##leer todos los renglones separados por enter
def df_text_to_string(df_text):
    df_text = df_text.applymap(str)
    string= ''
    for i, row in df_text.iterrows():
        line_text = ''.join(row)
        #print(line_text)
        string += line_text +'\n'
    return string
def list_dfs_to_string(list_df_text):
    String = ' '.join([df_text_to_string(x) for x in list_df_text])
    return String



def clean_text(s):
    ##acentos y caracteres especiales
    s = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", s), 0, re.I)
    s = normalize( 'NFC', s)
    #minusculas
    minus = s.lower()
    ##puntuacion
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in minus:
       if char not in punctuations:
           no_punct = no_punct + char

    return no_punct

def extract_dates(s):
    try:
        mesesDic = {'enero': '01',
                 'febrero': '02',
                 'marzo': '03',
                 'abril': '04',
                 'mayo': '05',
                 'junio': '06',
                 'julio': '07',
                 'agosto': '08',
                 'septiembre': '09',
                 'octubre': '10',
                 'noviembre': '11',
                 'diciembre': '12'}
        dates = re.finditer('(?P<day>[0-9]{2}) (de) (?P<month>[a-zA-Z]+) (de|del) (?P<year>[0-9]{4})',s)
        list_dates = []
        for x in dates:
            day = x.group('day')
            month = mesesDic[x.group('month')]
            year = x.group('year')
            fecha = datetime.date(int(year),int(month),int(day))
            list_dates.append(fecha)
        list_dates.sort(reverse=True)
        list_dates = [y.strftime('%d-%m-%Y') for y in list_dates]
           #print(list_dates)
        return list_dates[0]
    except:
        return None

def extract_unidades(s):
    try:
        unidades = re.finditer('((miles|millones) (de) (?P<moneda>(dolares|pesos|usd)( colombianos)?))',s,flags=2)
        i = 0
        for x in unidades:
            if i ==0:
                moneda1 = x.group()
            moneda = x.group('moneda')
            if moneda == 'usd':
                return x.group()
            if moneda == 'dolares':
                return x.group()

        return moneda1
    except:
        return None
    
def dates_in_tables_f(list_df_tables):
    dates_in_tables =[]
    for df_tables in list_df_tables:
        s = df_text_to_string(df_tables)
        date = extract_dates(s)
        
        dates_in_tables.append(date)
    dates_in_tables = [x for x in dates_in_tables if x]
    dates_in_tables.sort()
    date = dates_in_tables[-1]
    return date
def intenta_limpiar_texto(x):
    try:
        y=clean_text(x)
        return y
    except:
        return x

def loc_variable_key(list_df_tables,variable):
    lis_dict = []
    for hoja in range(len(list_df_tables)):
        
        for col in range(list_df_tables[hoja].shape[1]):
            #print(variable)
            for i in range(list_df_tables[hoja].shape[0]):
                    try:
                        s =list_df_tables[hoja].iloc[i,col]
                        #print(s)
                        var = re.search('|'.join(v_contables_dict[variable]),s)
                        if var:
                            d_ubi = {s:[hoja, col, i]}
                            lis_dict.append(d_ubi)
                            #print(s)
                            #print(hoja,i,col)
                    except:
                        pass
    return lis_dict



# Integración

#### Get aws analyze document

In [278]:
bucket = 'calacaschidas'
bucket_files = boto3.client('s3').list_objects(Bucket=bucket)['Contents']
bucket_files = [x['Key'] for x in bucket_files][1:]

bucket_file = 'Doc3.pdf'
file_path = os.path.join('DownTest', bucket_file)
if bucket_file not in os.listdir('DownTest'):
    boto3.client('s3').download_file(
        bucket, 
        file_path, 
        file_path
    )
list_df_tables, list_df_text = get_aws_analyze_document(file_path)

#### Unidades y fechas

In [279]:
Text = clean_text(list_dfs_to_string(list_df_text))

Unidades = extract_unidades(Text)
Fecha = extract_dates(Text)


In [280]:
##si no encontré esas variables, buscalas en las tablas 
if Fecha:
    print('va')
else:
    print('deja busco')
    ##busca fechas en las tablas
    Fecha = dates_in_tables_f(list_df_tables)
    
#if Unidades:
    
    

deja busco


#### Variables contables

In [281]:
##dic de los nombres posibles
v_contables_dict = {
    'Caja y Bancos':[
        'caja y bancos', 
        'efectivo y equivalentes en efectivo',
        'efectivo y equivalentes de efectivo',
    ],
    'Total activo':[
        'total activo',
        'suma de los activos', 
        'activo total'
    ],
    'Total pasivo':[
        'total pasivo',
        'suma de los pasivos',
        'pasivo total',
        'total pasivos'

    ],
    'Total patrimonio':[
        'total patrimonio',
        'suma de los patrimonios',
        'patrimonio total',
        'patrimonio',
        'suma del capital contable',
        'total capital contable', 
        'total capital', 
    ],
    'Ventas':[
        'ventas',  
        'ventas por operacion ordinacia',
        'ingresos por operacion',
        'ingresos por operacion ordinaria', 
        'ingresos operacionales', 
        'ventas brutas', 
        'ingreso por actividades ordinarias',
        'total ingresos operacionales'
    ],
    'Costo de ventas':[
        'costos de ventas', 
        'costos por ventas', 
        'costo de actividades ordinarias',
    ],
    'Utilidad Bruta':[
        'utilidad bruta',
        'perdida bruta',
    ],
    'utilidad operacional': [
        'utilidad operacional', 
        'perdida operacional',
    ],
    'utilidad antes de impuestos': [
        'utilidad antes de impuestos', 
        'perdida antes de impuestos',
    ],
    'utilidad neta': [
        'utilidad neta', 
        'perdida neta',
    ],
}

In [282]:
#df_clean_text = list_df_tables[0].applymap(intenta_limpiar)


In [283]:
#df_clean_text

In [284]:
#list_df_tables[0]

In [285]:
###limpieza del texto del df
for x in range(len(list_df_tables)):
    list_df_tables[x] = list_df_tables[x].applymap(intenta_limpiar)
    

In [286]:
v_contables_dict['Ventas']

['ventas',
 'ventas por operacion ordinacia',
 'ingresos por operacion',
 'ingresos por operacion ordinaria',
 'ingresos operacionales',
 'ventas brutas',
 'ingreso por actividades ordinarias',
 'total ingresos operacionales']

In [287]:
#list_df_tables[2].iteritems()

In [288]:
#s='total variable y costo de actividades ordinarias ventas jhhv hghvfh ñ'
#costo = re.search('|'.join(v_contables_dict['Costo de ventas']),s)

#[costo]

In [289]:
#print(costo)

In [290]:
#('|'.join(v_contables_dict['Costo de ventas']))

In [291]:
variables_contables ={}
for key in v_contables_dict:
    #print('***',key,'***')
    lis = loc_variable_key(list_df_tables,key)
    variables_contables[key] = lis
    #print('\n')
    ##arreglar jerarquia y quedarme con uno

In [292]:
variables_contables

{'Caja y Bancos': [{'efectivo y equivalentes de efectivo': [0, 1, 4]}],
 'Total activo': [{'total activos corrientes': [0, 1, 16]},
  {'total activos no corrientes': [0, 1, 25]},
  {'total activos': [0, 1, 26]}],
 'Total pasivo': [{'total pasivos corrientes': [1, 1, 16]},
  {'total pasivos no corrientes': [1, 1, 25]},
  {'total pasivos': [1, 1, 26]},
  {'total pasivos y patrimonio': [1, 1, 37]}],
 'Total patrimonio': [{'pasivos y patrimonio': [1, 1, 1]},
  {'patrimonio': [1, 1, 27]},
  {'total de patrimonio atribuible a los propietarios de la compania': [1,
    1,
    34]},
  {'total patrimonio': [1, 1, 36]},
  {'total pasivos y patrimonio': [1, 1, 37]}],
 'Ventas': [{'ingresos operacionales': [2, 1, 2]},
  {'total ingresos operacionales': [2, 1, 5]},
  {'gastos de ventas': [2, 1, 14]}],
 'Costo de ventas': [],
 'Utilidad Bruta': [],
 'utilidad operacional': [],
 'utilidad antes de impuestos': [],
 'utilidad neta': [{'perdida utilidad neta del periodo': [2, 1, 31]}]}

In [293]:
variables = variables_contables.copy()

In [294]:
variables['Fecha'] = Fecha
variables['Unidades'] = Unidades


In [295]:
variables

{'Caja y Bancos': [{'efectivo y equivalentes de efectivo': [0, 1, 4]}],
 'Total activo': [{'total activos corrientes': [0, 1, 16]},
  {'total activos no corrientes': [0, 1, 25]},
  {'total activos': [0, 1, 26]}],
 'Total pasivo': [{'total pasivos corrientes': [1, 1, 16]},
  {'total pasivos no corrientes': [1, 1, 25]},
  {'total pasivos': [1, 1, 26]},
  {'total pasivos y patrimonio': [1, 1, 37]}],
 'Total patrimonio': [{'pasivos y patrimonio': [1, 1, 1]},
  {'patrimonio': [1, 1, 27]},
  {'total de patrimonio atribuible a los propietarios de la compania': [1,
    1,
    34]},
  {'total patrimonio': [1, 1, 36]},
  {'total pasivos y patrimonio': [1, 1, 37]}],
 'Ventas': [{'ingresos operacionales': [2, 1, 2]},
  {'total ingresos operacionales': [2, 1, 5]},
  {'gastos de ventas': [2, 1, 14]}],
 'Costo de ventas': [],
 'Utilidad Bruta': [],
 'utilidad operacional': [],
 'utilidad antes de impuestos': [],
 'utilidad neta': [{'perdida utilidad neta del periodo': [2, 1, 31]}],
 'Fecha': '31-12-2

In [296]:
variables.keys()

dict_keys(['Caja y Bancos', 'Total activo', 'Total pasivo', 'Total patrimonio', 'Ventas', 'Costo de ventas', 'Utilidad Bruta', 'utilidad operacional', 'utilidad antes de impuestos', 'utilidad neta', 'Fecha', 'Unidades'])